In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install sentence_transformers
!pip install git+https://github.com/pmbaumgartner/setfit
!pip install git+https://github.com/LIAAD/yake


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pmbaumgartner/setfit to /tmp/pip-req-build-z3n4iqio
  Running command git clone -q https://github.com/pmbaumgartner/setfit /tmp/pip-req-build-z3n4iqio
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-qrwyfyi6
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-qrwyfyi6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/si

In [12]:
import torch
import random
import json
from pandas import Series, DataFrame
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import gzip
from transformers import pipeline
from random import choice, randint, shuffle
from pathlib import Path

pref = Path('/content/drive/MyDrive/WORK/data') 


tqdm.pandas()

list_words = ['project', 'proposal', 'job', 'solution', 'research',  'idea', 'algorithm', 'plan', 'initiative', 'announcement']

In [6]:
import sqlite3 as db
conn = db.connect('/content/drive/MyDrive/WORK/data/labels.db')
train_df = pd.read_sql_query("SELECT text,label FROM labels", conn)
train_df.sort_values("label", ascending=False).head()

,text,label
0,The suggested EA project / org: work to identi...,1
18,As a concrete program related to the safety of...,1
43,One possible area for exploration is around Sc...,1
40,I propose an infrastructure to generate more a...,1
58,"Hi Vaidehi,What kind of research do you plan o...",1


In [7]:
index = train_df.sort_values("label", ascending=False).index.to_list()

In [8]:
dataset = list(zip(train_df.text[index].to_list()[:20], train_df.label[index].to_list()))
shuffle(dataset)
docs = [i[0] for i in dataset] 
labels = [i[1] for i in dataset]

In [13]:
classifier = pipeline("zero-shot-classification", model="facebook/xglm-564M")


Downloading:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/276 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [19]:
classifier_1 = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
val_com = classifier('The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', list_words)
val_sep = classifier('The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', list_words, multi_label=True)
print(val_com)
print(val_sep)


{'sequence': 'The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', 'labels': ['algorithm', 'job', 'initiative', 'announcement', 'proposal', 'research', 'idea', 'plan', 'solution', 'project'], 'scores': [0.1151251271367073, 0.11135178804397583, 0.10781940817832947, 0.10668908804655075, 0.10451661795377731, 0.10134973376989365, 0.10125191509723663, 0.09190346300601959, 0.0835309699177742, 0.07646188884973526]}
{'sequence': 'The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', 'labels': ['algorithm', 'job', 'initiative', 'announcement', 'proposal', 'research', 'idea', 'plan', 'solution', 'project'], 'scores': [0.8512539267539978, 0.8469846844673157, 0.842759907245636, 0.8413583040237427, 0.8385930061340332, 0.8343847990036011, 0.834251344203949, 0.820419073

In [20]:
val_com_1 = classifier_1('The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', list_words)
val_sep_1 = classifier_1('The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', list_words, multi_label=True)
print(val_com_1)
print(val_sep_1)


{'sequence': 'The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', 'labels': ['project', 'proposal', 'plan', 'idea', 'job', 'initiative', 'research', 'solution', 'announcement', 'algorithm'], 'scores': [0.2457607388496399, 0.1895020455121994, 0.16583526134490967, 0.12890934944152832, 0.08682485669851303, 0.06656523793935776, 0.04413896054029465, 0.033563096076250076, 0.026474149897694588, 0.012426274828612804]}
{'sequence': 'The suggested EA project / org: work to identify important, simple, and uncontroversial ideas as they arise and use social media to get people to pay attention to them.', 'labels': ['proposal', 'project', 'plan', 'idea', 'research', 'initiative', 'announcement', 'job', 'solution', 'algorithm'], 'scores': [0.83073490858078, 0.8001330494880676, 0.7185404896736145, 0.7105913758277893, 0.49306896328926086, 0.47426173090934753, 0.4472229480743408, 0.4319

**Посмотрим на результаты применения этих классификаторов к размеченному набору из 20 текстов (8 поз и 12 нег)**

In [26]:
results = []
for items in dataset:
  res = classifier(items[0], list_words)
  res_ = classifier(items[0], list_words, multi_label=True)
  res_1 = classifier_1(items[0], list_words)
  res_1_ = classifier_1(items[0], list_words, multi_label=True)
  print(items[0])
  results.append((res['labels'][0], res['scores'][0],  res_['labels'][0], res_['scores'][0], res_1['labels'][0], res_1['scores'][0], res_1_['labels'][0], res_1_['scores'][0], 'label=', items[1]))
  print(results[-1])


2) What role do you see entrepreneurs (vs. established organizations) play in this field, including incubation programs like CharityEntrepreneurship.com that has incubated mental health charities before?3) How do you assess the potential of new mental health treatments for the Global South? Is this sufficiently prioritized and do you see particular roadblocks to rapid adoption?
('job', 0.12226090580224991, 'job', 0.8760474324226379, 'initiative', 0.3546917736530304, 'initiative', 0.7312790751457214, 'label=', 0)
Personally I believe that we should go even further, and look into using assurance contracts to help create "affinity cities" and zoom-towns based on common interests -- we should create new EA hubs in well-chosen parts of the USA, then when people move there we can experiment with various kinds of community support (childcare, etc) and exciting new forms of community governance/decisionmaking (maybe all the EAs who use a coworking space pay a fee that gets spent on community-i

**Один из вариантов дальнейшего развитие этого подхода:**

---


Обучить классификатор на полученных результатах применения zero-shot к размеченному набору для одной из этих моделей, а затем применить его к исходному набору комментариев, с соотвествующей моделью

Но с составом набора признаков, видимо, повозиться придется